In [6]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

In [417]:
potential = 'QHO'
cutoff = 16
shots = 1024

In [418]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [419]:
eigenvalues

array([ 0.+0.j,  1.+0.j,  1.+0.j,  2.+0.j,  2.+0.j,  3.+0.j,  3.+0.j,
        4.+0.j,  4.+0.j,  5.+0.j,  5.+0.j,  6.+0.j,  6.+0.j,  7.+0.j,
        7.+0.j,  7.+0.j,  8.+0.j,  8.+0.j,  8.+0.j,  9.+0.j,  9.+0.j,
       10.+0.j, 10.+0.j, 11.+0.j, 11.+0.j, 12.+0.j, 12.+0.j, 13.+0.j,
       13.+0.j, 14.+0.j, 14.+0.j, 15.+0.j])

In [420]:
min_eigenvalue

np.float64(0.0)

In [421]:
num_qubits

5

In [422]:
hamiltonian

SparsePauliOp(['IIIII', 'IIIZI', 'IIIZZ', 'IIZII', 'IIZIZ', 'IIZZI', 'IIZZZ', 'IZIII', 'IZIIZ', 'IZIZI', 'IZIZZ', 'IZZII', 'IZZIZ', 'IZZZI', 'IZZZZ', 'ZIIII'],
              coeffs=[ 7.5+0.j, -0.5+0.j, -0.5+0.j, -1.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j,
 -3.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j, -0.5+0.j,  0.5+0.j,  0.5+0.j,
 -0.5+0.j,  0.5+0.j])

In [423]:
operator_pool = [
    qml.Rot(0.0, 0.0, 0.0, wires=0),
    qml.Rot(0.0, 0.0, 0.0, wires=1)
    #parameterized_CNOT(param=0.0)
    #qml.CZ(wires=[0,1])
]

In [429]:
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def circuit():
    qml.PauliX(0)
    qml.PauliX(1)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [430]:
print(qml.draw(circuit)())

0: ──X─┤ ╭<𝓗(M0)>
1: ──X─┤ ├<𝓗(M0)>
2: ────┤ ├<𝓗(M0)>
3: ────┤ ├<𝓗(M0)>
4: ────┤ ╰<𝓗(M0)>

M0 = 
[[ 1.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  2.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  3.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ... 13.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j 14.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  7.+0.j]]


In [431]:
opt = qml.AdaptiveOptimizer()

for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool, drain_pool=True)
    if i % 3 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        print(qml.draw(circuit, decimals=None, show_matrices=False)())
        print()
    if energy < 1e-5:
        break

n = 0,  E = 8.00000000 H, Largest Gradient = 0.000
0: ──X──Rot─┤ ╭<𝓗(M0)>
1: ──X──────┤ ├<𝓗(M0)>
2: ─────────┤ ├<𝓗(M0)>
3: ─────────┤ ├<𝓗(M0)>
4: ─────────┤ ╰<𝓗(M0)>



In [122]:
# Pool of operators (e.g., single-qubit rotations and entangling gates)
operator_pool = [
    qml.Rot(0.0, 0.0, 0.0, wires=0)
    #qml.Rot(0.0, 0.0, 0.0, wires=1)
    #qml.CNOT(wires=[0, 1])
    #qml.CZ(wires=[0,1])
]

In [124]:
# Define the device
dev = qml.device("default.qubit", wires=num_qubits)

# Adaptive ansatz, starts with the basis state and grows dynamically
def adaptive_ansatz(params, ops):

    # basis state
    for i in range(num_qubits):
        qml.PauliX(i)

    for param, op in zip(params, ops):
        if len(param) == 3:  # For rotation gates
            qml.Rot(*param, wires=op.wires)
        else:  # For non-parameterized gates
            op

    #return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [112]:
# Cost function to minimize the expectation value of the Hamiltonian
def cost_function(params, ops):
    
    @qml.qnode(dev)
    def circuit():
        adaptive_ansatz(params, ops)
        return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    
    return circuit()

In [212]:
def compute_operator_gradients(params, ops):
    grads = []
    for i, op in enumerate(operator_pool):
        # Define a QNode for gradient computation specific to the operator
        @qml.qnode(dev)
        def gradient_circuit(params):
            adaptive_ansatz(params, ops + [op])
            return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

        try:
            # Compute the gradient of the energy with respect to the operator
            grad_fn = qml.grad(gradient_circuit)
            grad = grad_fn(params)
            grads.append(grad)
        except Exception as e:
            print(f"Error computing gradient for operator {i}: {e}")
            grads.append(0.0)  # Append zero if gradient computation fails

    print(f"Gradients: {grads}")
    return np.abs(np.array(grads))


In [ ]:
# Main ADAPT-VQE routine
tol=1e-4,
max_steps=50

params = []
ops = []
opt = qml.AdamOptimizer(stepsize=0.1)
step = 0

while step < max_steps:

    if not params:  # For the first operator, initialize a single set of parameters
        params = [pnp.tensor(np.zeros(3), requires_grad=True)]
    else:  # For subsequent operators, extend the parameter list
        params.append(pnp.tensor(np.zeros(3), requires_grad=True))

    # Compute operator gradients
    grads = compute_operator_gradients(params, ops)

    # Select the operator with the largest gradient
    max_grad_idx = np.argmax(grads)
    if grads[max_grad_idx] < tol:
        print("Converged!")
        break

    # Add the selected operator to the ansatz
    ops.append(operator_pool[max_grad_idx])
    params.append(np.zeros(3))  # Initialize parameters for the new gate

    # Optimize the parameters
    params = opt.step(lambda p: cost_function(p, ops), params)

    # Print the current energy
    energy = cost_function(params, ops)
    print(f"Step {step + 1}: Energy = {energy:.6f}")
    step += 1

In [235]:
params

[tensor([0., 0., 0.], requires_grad=True)]

In [240]:
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def gradient_circuit(params, ops_list):
    adaptive_ansatz(params, ops_list)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [288]:
grads = []

for i, op in enumerate(operator_pool):
    # Define a QNode for gradient computation specific to the operator
    ops_list = ops + [op]
    print(ops_list)
    try:
        # Compute the gradient of the energy with respect to the operator
        #grad = qml.grad(gradient_circuit)(params, ops_list)
        grad = qml.grad(adaptive_ansatz)(params, ops_list)
        print(params)
        #grad = grad_fn(params)
        grads.append(grad)
        print("hello")
        grads
    except Exception as e:
        print(f"Error computing gradient for operator {i}: {e}")
        grads.append(0.0)

[Rot(0.0, 0.0, 0.0, wires=[0])]
[tensor([0., 0., 0.], requires_grad=True)]
hello


In [290]:
grads

[()]

In [291]:
params

[tensor([0., 0., 0.], requires_grad=True)]

In [244]:
ops_list

[Rot(0.0, 0.0, 0.0, wires=[0])]

In [257]:
for param, op in zip(params, ops_list):
    print(param)
    print(op)

[0. 0. 0.]
Rot(0.0, 0.0, 0.0, wires=[0])


In [263]:
pnp.tensor(param, requires_grad=True)

tensor([0., 0., 0.], requires_grad=True)

In [283]:
# Define the device
dev = qml.device("default.qubit", wires=num_qubits)

# Adaptive ansatz, starts with the basis state and grows dynamically
@qml.qnode(dev)
def adaptive_ansatz(params, ops):

    # basis state
    for i in range(num_qubits):
        qml.PauliX(i)

    for param, op in zip(params, ops):
        gate_params = pnp.tensor([0.0,0.0,0.0], requires_grad=True)
        if len(param) == 3:  # For rotation gates
            qml.Rot(0.0, 0.0, 0.0, wires=op.wires)
        else:  # For non-parameterized gates
            op

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [121]:
print("Final Parameters:", params)
print("Final Operators:", ops)

Error computing gradient for operator 0: multi-dimensional sub-views are not implemented
Error computing gradient for operator 1: multi-dimensional sub-views are not implemented
Gradients: [0.0, 0.0]
Converged!
Final Parameters: []
Final Operators: []
